In [2]:
! pip install sentence_transformers==2.7.0


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [5]:
from sentence_transformers import SentenceTransformer
import numpy as np

/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Q1. Getting the embeddings model

Create the embedding for this user question: 

```user_question = "I just discovered the course. Can I still join it?"```


In [17]:
model = SentenceTransformer('multi-qa-distilbert-cos-v1')
user_question = "I just discovered the course. Can I still join it?"
v = model.encode(user_question)
v[:5]

array([ 0.07822265, -0.04013114,  0.03861359, -0.00017897,  0.08923471],
      dtype=float32)

In [8]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

#### Q2. Creating the embeddings
Now for each document, we will create an embedding for both question and answer fields.

We want to put all of them into a single matrix X:

- Create a list embeddings
- Iterate over each document
- qa_text = f'{question} {text}'
- compute the embedding for qa_text, append to embeddings
- At the end, let X = np.array(embeddings) (import numpy as np)
What's the shape of X? (X.shape). Include the parantheses.

In [9]:
# We will use only a subset of the questions - the questions for "machine-learning-zoomcamp". After filtering, you should have only 375 documents
documents_subset = [doc for doc in documents if doc["course"]=="machine-learning-zoomcamp"]

In [10]:
for d in documents_subset:
    qa_text = f'{d["question"]} {d["text"]}'


In [19]:
qa_text

'Any advice for adding the Machine Learning Zoomcamp experience to your LinkedIn profile? I’ve seen LinkedIn users list DataTalksClub as Experience with titles as:\nMachine Learning Fellow\nMachine Learning Student\nMachine Learning Participant\nMachine Learning Trainee\nPlease note it is best advised that you do not list the experience as an official “job” or “internship” experience since DataTalksClub did not hire you, nor financially compensate you.\nOther ways you can incorporate the experience in the following sections:\nOrganizations\nProjects\nSkills\nFeatured\nOriginal posts\nCertifications\nCourses\nBy Annaliese Bronz\nInteresting question, I put the link of my project into my CV as showcase and make posts to show my progress.\nBy Ani Mkrtumyan'

In [11]:
embeddings = model.encode(qa_text)

In [12]:
X = np.array(embeddings)
X.shape

(768,)

#### Q3. Search

We have the embeddings and the query vector. Now let's compute the cosine similarity between the vector from Q1 (let's call it v) and the matrix from Q2.

The vectors returned from the embedding model are already normalized (you can check it by computing a dot product of a vector with itself - it should return something very close to 1.0). This means that in order to compute the coside similarity, it's sufficient to multiply the matrix X by the vector v:

In [13]:
embeddings.dot(v)

0.25212276

#### Q4. Hit-rate for our search engine

Let's evaluate the performance of our own search engine. We will use the hitrate metric for evaluation.



In [14]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [21]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(v, num_results=5)

[{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'}]

In [24]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

hit_rate(search_engine.search(v, num_results=5))


0.0